In [ ]:
import sys
sys.executable

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import fastapi
import uvicorn
import dotenv

print("OK")

In [ ]:
import sys
print(sys.executable)

In [ ]:
import pandas, matplotlib, seaborn, sklearn, fastapi, uvicorn, dotenv
print("ok")

In [ ]:
import os

print(os.getcwd())
print(os.listdir())

In [ ]:
import os

os.listdir("../data")

In [ ]:
import os

os.listdir("../data/processed")

In [ ]:
import pandas as pd

df = pd.read_csv("../data/processed/final_data_clean.csv")
df.shape

In [ ]:
df.head(2)

In [ ]:
import pandas as pd

# 1) decide how many tag columns to create
n_tags = int(df["tag_count"].max())  # uses your existing tag_count column

# 2) split and expand into exactly n_tags columns
tags_expanded = (
    df["tags_str"]
      .fillna("")                              # handle NaNs
      .str.split(r"\s*,\s*", expand=True)      # split on commas with optional spaces
      .reindex(columns=range(n_tags))          # force exactly n_tags columns (pads with NaN)
)

# 3) clean up empty strings to proper missing values
tags_expanded = tags_expanded.replace("", pd.NA)

# 4) assign back onto the SAME df (no dropping)
df[[f"tag_{i}" for i in range(n_tags)]] = tags_expanded
df.head(2)

In [ ]:
import pandas as pd

# number of tag columns to create
n_tags = int(df["tag_count"].max())

# split tags
tags_expanded = (
    df["tags_str"]
      .fillna("")
      .str.split(r"\s*,\s*", expand=True)
      .reindex(columns=range(n_tags))
      .replace("", pd.NA)
)

# assign back to the SAME dataframe, starting from tag_1
df[[f"tag_{i}" for i in range(1, n_tags + 1)]] = tags_expanded.values

df.head(2)

In [ ]:
df.drop(columns=["tag_0"], inplace=True)
df.head(2)

In [ ]:
import pandas as pd

tag_cols = df.filter(regex=r"^tag_\d+$").columns

unique_tags = (
    pd.Series(df[tag_cols].values.ravel())
      .dropna()
      .astype(str)
      .sort_values()
      .unique()
)

len(unique_tags), unique_tags

In [ ]:
missing_tags_rows = df["tags_str"].isna().sum()
missing_tags_rows

In [ ]:
missing_genres_rows = (
    df["genres"].isna() |
    df["genres"].str.strip().eq("")
).sum()

missing_genres_rows

In [ ]:
# Define missing and basic counts
import pandas as pd
import numpy as np

# robust missing definition
missing_tags_mask = df["tags_str"].isna() | df["tags_str"].astype(str).str.strip().eq("")

n_missing = int(missing_tags_mask.sum())
n_total = len(df)

print(f"Missing tags_str rows: {n_missing} / {n_total} ({n_missing/n_total:.2%})")

In [ ]:
# Check if those authors have other books with tags filled
# authors that appear in missing-tag rows
authors_with_missing = set(df.loc[missing_tags_mask, "author"].dropna().unique())

# authors that have at least one non-missing tags_str row
authors_with_tags = set(df.loc[~missing_tags_mask, "author"].dropna().unique())

# overlap: imputation possible via author
authors_overlap = authors_with_missing & authors_with_tags

print(f"Authors in missing rows: {len(authors_with_missing)}")
print(f"Authors with at least one filled tags_str: {len(authors_with_tags)}")
print(f"Missing-row authors that have other tagged books (imputable by author): {len(authors_overlap)}")

# how many missing rows are imputable by author?
imputable_rows = df.loc[missing_tags_mask, "author"].isin(authors_overlap).sum()
print(f"Missing rows imputable via author overlap: {int(imputable_rows)} / {n_missing} ({imputable_rows/n_missing:.2%})")

In [ ]:
# Build an author → candidate tags pool (from non-missing rows)
non_missing = df.loc[~missing_tags_mask, ["author", "tags_str", "rating_count"]].copy()

# normalize tags_str formatting (optional but helps reduce near-duplicates)
non_missing["tags_str_norm"] = (
    non_missing["tags_str"].astype(str)
    .str.split(r"\s*,\s*")
    .apply(lambda xs: ", ".join([x.strip() for x in xs if x.strip()]))
)

# build pool: for each author, collect unique candidate tag strings, with support metric
author_pool = (
    non_missing
    .groupby("author", dropna=True)
    .apply(lambda g: g[["tags_str_norm", "rating_count"]]
           .drop_duplicates("tags_str_norm")
           .sort_values("rating_count", ascending=False))
)

# sanity peek for one author
# display(author_pool.loc["Suzanne Collins"].head(10))
print("Author pool built.")

In [ ]:
# for each author, take the candidate tags from their most rated book (highest rating_count). This is a reasonable proxy
# collapse pool to a single "best" tags_str per author
author_best_tags = (
    non_missing
    .sort_values("rating_count", ascending=False)
    .drop_duplicates("author")
    .set_index("author")["tags_str_norm"]
)

print(f"Authors with a chosen best tags_str: {author_best_tags.index.nunique()}")

In [ ]:
# Impute missing tags_str and keep provenance
df["tags_imputed_from_author"] = pd.NA  # provenance column

# fill only missing
to_fill = missing_tags_mask & df["author"].isin(author_best_tags.index)

df.loc[to_fill, "tags_imputed_from_author"] = df.loc[to_fill, "author"]
df.loc[to_fill, "tags_str"] = df.loc[to_fill, "author"].map(author_best_tags)

print(f"Filled tags_str for rows: {int(to_fill.sum())}")
print(f"Remaining missing tags_str: {int((df['tags_str'].isna() | df['tags_str'].astype(str).str.strip().eq('')).sum())}")

In [ ]:
# quick QA sample
# show a sample of filled rows
df.loc[to_fill, ["title", "author", "tags_str", "tags_imputed_from_author"]].sample(
    n=min(10, int(to_fill.sum())), random_state=42
)

In [ ]:
missing_tags_rows = df["tags_str"].isna().sum()
missing_tags_rows

In [ ]:
# define empty / missing tags_str
empty_tags_mask = (
    df["tags_str"].isna() |
    df["tags_str"].astype(str).str.strip().eq("")
)

# create filtered copy
df_post_drop = df.loc[~empty_tags_mask].copy()

# sanity check
len(df), len(df_post_drop)

In [ ]:
# reseting the index 
df_post_drop.reset_index(drop=True, inplace=True)

In [ ]:
df_post_drop.head(5)

In [ ]:
# Creating a wieghted attribution model for clustering tags into moods, and scoring the books per mood.

In [ ]:
# Define the frozen weight map
MOOD_TAG_WEIGHTS = {
    "bedtime": {
        "emotional": 1.0,
        "romantic": 0.9,
        "fantasy": 0.6,
    },
    "beach": {
        "easy-read": 1.0,
        "light": 0.9,
        "travel": 0.8,
        "nature": 0.8,
        "emotional": 0.6,
        "romantic": 0.6,
    },
    "easy": {
        "easy-read": 1.0,
        "short": 0.9,
        "fast-paced": 0.8,
    },
    "on_the_go": {
        "fast-paced": 1.0,
        "suspenseful": 0.9,
        "travel": 0.7,
        "nature": 0.5,
    },
    "educational": {
        "educational": 1.0,
        "historical": 0.9,
        "real-life": 0.9,
        "deep": 0.8,
        "literary": 0.7,
        "classic": 0.6,
        "poetic": 0.5,
    },
    "adrenaline": {
        "suspenseful": 1.0,
        "dark": 0.9,
        "psychological": 0.9,
        "fast-paced": 0.8,
        "sci-fi": 0.6,
        "fantasy": 0.4,
    },
}

In [ ]:
# Collect tag columns and normalize to sets
import pandas as pd
import numpy as np

tag_cols = df_post_drop.filter(regex=r"^tag_\d+$").columns

df_post_drop["tag_set"] = df_post_drop[tag_cols] \
    .apply(lambda row: {t for t in row if pd.notna(t)}, axis=1)

In [ ]:
# Average weighted match
def avg_weighted_match(tag_set, mood_weights):
    matched = tag_set.intersection(mood_weights.keys())
    if not matched:
        return 0.0
    return np.mean([mood_weights[t] for t in matched])

In [ ]:
# Broad (saturated) score
def broad_score(tag_set, mood_weights, K=2):
    matched = tag_set.intersection(mood_weights.keys())
    if not matched:
        return 0.0
    return sum(mood_weights[t] for t in matched) / min(K, len(matched))

In [ ]:
# Compute all mood scores
for mood, weights in MOOD_TAG_WEIGHTS.items():
    df_post_drop[f"score_{mood}"] = df_post_drop["tag_set"] \
        .apply(lambda tags: avg_weighted_match(tags, weights))

    df_post_drop[f"score_{mood}_broad"] = df_post_drop["tag_set"] \
        .apply(lambda tags: broad_score(tags, weights))

In [ ]:
# Top mood + margin (for strict mode)
score_cols = [f"score_{m}" for m in MOOD_TAG_WEIGHTS.keys()]

df_post_drop["top_mood"] = df_post_drop[score_cols].idxmax(axis=1)
df_post_drop["top_mood_score"] = df_post_drop[score_cols].max(axis=1)

df_post_drop["second_mood_score"] = (
    df_post_drop[score_cols]
    .apply(lambda r: r.nlargest(2).iloc[-1], axis=1)
)

df_post_drop["top_mood_margin"] = (
    df_post_drop["top_mood_score"] - df_post_drop["second_mood_score"]
)

In [ ]:
# Example filters (for later Streamlit wiring)
df_balanced = df_post_drop.sort_values("score_adrenaline", ascending=False)

In [ ]:
# Best-only (strict, with margin)
MARGIN = 0.1

df_strict = df_post_drop[
    (df_post_drop["top_mood"] == "score_adrenaline") &
    (df_post_drop["top_mood_margin"] >= MARGIN)
].sort_values("score_adrenaline", ascending=False)

In [ ]:
# Broad
df_broad = df_post_drop.sort_values("score_adrenaline_broad", ascending=False)

In [ ]:
df_post_drop.columns.tolist()

In [ ]:
df_post_drop[
    [
        "title",
        "author",
        "tags_str",
        "tag_set",
        "score_bedtime",
        "score_beach",
        "score_easy",
        "score_on_the_go",
        "score_educational",
        "score_adrenaline",
        "top_mood",
        "top_mood_margin",
    ]
].head(10)

In [ ]:
df_post_drop.sample(1, random_state=42)[
    [
        "title",
        "tags_str",
        "tag_set",
        "score_bedtime",
        "score_beach",
        "score_easy",
        "score_on_the_go",
        "score_educational",
        "score_adrenaline",
        "top_mood",
    ]
]

In [ ]:
df_post_drop["tag_set"] = (
    df_post_drop["tags_str"]
    .astype(str)
    .str.split(r"\s*,\s*")
    .apply(lambda xs: {x for x in xs if x})
)

In [ ]:
for mood, weights in MOOD_TAG_WEIGHTS.items():
    df_post_drop[f"score_{mood}"] = df_post_drop["tag_set"] \
        .apply(lambda tags: avg_weighted_match(tags, weights))

In [ ]:
df_post_drop.loc[786, [
    "title",
    "tags_str",
    "tag_set",
    "score_bedtime",
    "score_easy",
    "score_educational",
    "score_adrenaline",
    "top_mood",
]]

In [ ]:
score_cols = [f"score_{m}" for m in MOOD_TAG_WEIGHTS.keys()]

df_post_drop["top_mood"] = df_post_drop[score_cols].idxmax(axis=1)
df_post_drop["top_mood_score"] = df_post_drop[score_cols].max(axis=1)

In [ ]:
df_post_drop.loc[786, [
    "title",
    "score_easy",
    "score_educational",
    "score_adrenaline",
    "top_mood",
    "top_mood_score",
]]

In [ ]:
df_post_drop.sample(1, random_state=42)[
    [
        "title",
        "tags_str",
        "tag_set",
        "score_bedtime",
        "score_beach",
        "score_easy",
        "score_on_the_go",
        "score_educational",
        "score_adrenaline",
        "top_mood",
        "top_mood_score",
    ]
]

In [ ]:
# Strictness filtering

In [ ]:
# recompute margin
score_cols = [f"score_{m}" for m in MOOD_TAG_WEIGHTS.keys()]

df_post_drop["second_mood_score"] = (
    df_post_drop[score_cols]
    .apply(lambda r: r.nlargest(2).iloc[-1], axis=1)
)

df_post_drop["top_mood_margin"] = (
    df_post_drop["top_mood_score"] - df_post_drop["second_mood_score"]
)

In [ ]:
# strict filter for a selected mood
MARGIN = 0.1
SELECTED_MOOD = "score_adrenaline"

df_strict = df_post_drop[
    (df_post_drop["top_mood"] == SELECTED_MOOD) &
    (df_post_drop["top_mood_margin"] >= MARGIN)
].sort_values("top_mood_score", ascending=False)

In [ ]:
# quick sanity check
df_strict[
    ["title", "top_mood", "top_mood_score", "top_mood_margin"]
].head(10)

In [ ]:
# Add a minimum score threshold
MIN_SCORE = 0.8
MARGIN = 0.1
SELECTED_MOOD = "score_adrenaline"

df_strict = df_post_drop[
    (df_post_drop["top_mood"] == SELECTED_MOOD) &
    (df_post_drop["top_mood_margin"] >= MARGIN) &
    (df_post_drop["top_mood_score"] >= MIN_SCORE)
].sort_values("top_mood_score", ascending=False)

In [ ]:
# For non-strict modes
df_filtered = df_post_drop[
    df_post_drop["score_adrenaline"] >= MIN_SCORE
].sort_values("score_adrenaline", ascending=False)

In [ ]:
df_post_drop.columns.tolist()